In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

train_original = pd.read_csv("train.csv")
test_original = pd.read_csv("test.csv")

train = train_original.copy()
test = test_original.copy()

print(train.head())
print(train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
  

In [4]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,NaN,0.000000,0.000000,NaN
50%,1100.500000,3.000000,NaN,0.000000,0.000000,NaN
75%,1204.750000,3.000000,NaN,1.000000,0.000000,NaN
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [29]:
# Fill the NaN values of Age
# Generate random values between mean + std and mean - std

train_mean = train["Age"].mean()
train_std = train["Age"].std()
train_count = train["Age"].isnull().sum()

rand_train = np.random.randint(train_mean - train_std, train_mean + train_std, size = train_count)

train["Age"][np.isnan(train["Age"])] = rand_train

# Similarly for Test

test_mean = test["Age"].mean()
test_std = test["Age"].std()
test_count = test["Age"].isnull().sum()

rand_test = np.random.randint(test_mean - test_std, test_mean + test_std, size = test_count)

test["Age"][np.isnan(test["Age"])] = rand_test

C:\Users\Aakanksha\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Aakanksha\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
train["Embarked"].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [20]:
# We replace Nan values in Embraked
# With most frequently occuring value 'S'

train["Embarked"] = train["Embarked"].fillna("S")

In [23]:
test["Embarked"].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [24]:
# Again repeat the replacement procedure

test["Embarked"] = test["Embarked"].fillna("S")

In [28]:
# Convert all strings to integers

# Starting with Embarked

train["Embarked"].loc[train["Embarked"] == "S"] = 0
train["Embarked"].loc[train["Embarked"] == "C"] = 1
train["Embarked"].loc[train["Embarked"] == "Q"] = 2

test["Embarked"].loc[test["Embarked"] == "S"] = 0
test["Embarked"].loc[test["Embarked"] == "C"] = 1
test["Embarked"].loc[test["Embarked"] == "Q"] = 2

In [27]:
# Conversion of "Sex" to numerical values

train["Sex"].loc[train["Sex"] == "male"] = 0
train["Sex"].loc[train["Sex"] == "female"] = 1

test["Sex"].loc[test["Sex"] == "male"] = 0
test["Sex"].loc[test["Sex"] == "female"] = 1

In [31]:
# Convert SibSp and Parch into one "Family" attribue
# Value 0 for no family on board
# Value 1 for one or more family members on board

train["Family"] = train["SibSp"] + train["Parch"]
test["Family"] = test["SibSp"] + test["Parch"]

train["Family"].loc[train["Family"] > 0] = 1
train["Family"].loc[train["Family"] == 0] = 0

test["Family"].loc[test["Family"] > 0] = 1
test["Family"].loc[test["Family"] == 0] = 0

In [39]:
# Convert Age into Age-Brackets
# Assign numbers to those brackets

for val in train["Age"]:
    if val >= 0 and val < 20:
        train["Age"] = 0
    elif val >= 20 and val < 40:
        train["Age"] = 1
    elif val >= 40 and val < 60:
        train["Age"] = 2
    else:
        train["Age"] = 3

In [40]:
for val in test["Age"]:
    if val >= 0 and val < 20:
        test["Age"] = 0
    elif val >= 20 and val < 40:
        test["Age"] = 1
    elif val >= 40 and val < 60:
        test["Age"] = 2
    else:
        test["Age"] = 3

In [43]:
train = train.drop(["PassengerId", "Name", "SibSp", "Parch", "Ticket", "Fare", "Cabin"], axis = 1)

In [46]:
#Use the Random Forest Classifier

# from sklearn.ensemble import RandomForestClassifier

In [47]:
train_x = train.drop("Survived", axis =1)

In [48]:
train_y = train["Survived"]

In [63]:
test_x = test.drop(["PassengerId", "Name", "SibSp", "Parch", "Cabin", "Ticket", "Fare"], axis = 1)

In [65]:
train_x.shape, train_y.shape, test_x.shape

((891, 5), (891L,), (418, 5))

In [66]:
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest.fit(train_x, train_y)
random_predict = random_forest.predict(test_x)

random_forest.score(train_x, train_y)

acc_random_forest = round(random_forest.score(train_x, train_y) * 100, 2)
print acc_random_forest

81.14
